<a href="https://colab.research.google.com/github/aliyyah-u/NLP_Medical_NER/blob/main/NLP_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
__author__ = "Aliyyah Uddin"
__version__ = "IN3045 City, University of London, Spring 2025"

NLP for clinical tasks

NLP for essentially clinical tasks. So you have lots of data. It comes and people want to make quick decisions given sentences of this kind. So they have to identify has something happened to someone. What is that something. Okay. So this says the baby had cardiac arrest at 9 a.m.. So what are the entities here that potentially are going to be useful? 9 a.m. and the term cardiac arrest, which is sort of a named entity. It's a it's a kind of a procedure, or it's a kind of a pathology that clinicians are interested in. You see what I did? I did not only consider named entities of a form where they are all capitalised. So these are also a form of named entities. These are domain specific.

https://huggingface.co/datasets/parsa-mhmdi/Medical_NER/viewer/default/train?row=46

? Create a NER tagger for Swedish medical documents

In [3]:
pip install --upgrade huggingface_hub

In [4]:
from huggingface_hub import login
login()

In [6]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("hf://datasets/parsa-mhmdi/Medical_NER/data/train-00000-of-00001.parquet")